## GridWorld
Ph.D Leonarod A, Espinosa, M.Sc Andrej Scherbakov-Parland, BIT Kristoffer Kuvaja Adolfsson

### Bibliography:

* Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. MIT press, 2018.
http://incompleteideas.net/book/bookdraft2017nov5.pdf  (chapter 4)

In [1]:
# imports
import numpy as np

In [2]:
# Utilits
def printV(V, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):
            state = grid.m * idx + idy
            print('%.2f' % V[state], end='\t')
        print('\n')
    print('--------------------')

def printPolicy(policy, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):
            state = grid.m * idx + idy
            if state in grid.stateSpace:
                string = ''.join(policy[state])
                print(string, end='\t')
            else:
                print('', end='\t')
        print('\n')
    print('--------------------')

def printQ(Q, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):
            state = grid.m * idx + idy
            if state != grid.m * grid.n - 1:
                vals = [np.round(Q[state,action], 5) for action in grid.possibleActions]
                print(vals, end='\t')
        print('\n')
    print('--------------------')

def sampleReducedActionSpace(grid, action):
    actions = grid.possibleActions[:]
    actions.remove(action)
    sample = np.random.choice(actions)
    return sample

In [3]:
class WindyGrid(object):
    def __init__(self, m, n, wind):
        self.grid = np.zeros((m,n))                            # representation of the grid
        self.m = m
        self.n = n
        self.stateSpace = [i for i in range(self.m*self.n)]
        self.stateSpace.remove(28)                              # Terminal state
        self.stateSpacePlus = [i for i in range(self.m*self.n)] # State space + terminal state
        self.actionSpace = {'U': -self.m, 'D': self.m,
                            'L': -1, 'R': 1}
        self.possibleActions = ['U', 'D', 'L', 'R']
        self.agentPosition = 0
        self.wind = wind

    def isTerminalState(self, state):
        return state in self.stateSpacePlus and state not in self.stateSpace

    def getAgentRowAndColumn(self):                               # position of agent
        x = self.agentPosition // self.m
        y = self.agentPosition % self.n
        return x, y

    def setState(self, state):
        x, y = self.getAgentRowAndColumn()
        self.grid[x][y] = 0
        self.agentPosition = state
        x, y = self.getAgentRowAndColumn()
        self.grid[x][y] = 1

    def offGridMove(self, newState, oldState):
        # if we move into a row not in the grid
        if newState not in self.stateSpacePlus:
            return True
        # if we're trying to wrap around to next row
        elif oldState % self.m == 0 and newState  % self.m == self.m - 1:
            return True
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        else:
            return False

    # Include wind stenght.
    def step(self, action):
        agentX, agentY = self.getAgentRowAndColumn()
        if agentX > 0:
            resultingState = self.agentPosition + self.actionSpace[action] + \
                            self.wind[agentY] * self.actionSpace['U']
            if resultingState < 0: #if the wind is trying to push agent off grid
                resultingState += self.m
        else:
            if action == 'L' or action == 'R':
                resultingState = self.agentPosition + self.actionSpace[action]
            else:
                resultingState = self.agentPosition + self.actionSpace[action] + \
                            self.wind[agentY] * self.actionSpace['U']
        #reward = -1 if not self.isTerminalState(resultingState) else 0
        reward = -1
        if not self.offGridMove(resultingState, self.agentPosition):
            self.setState(resultingState)
            return resultingState, reward, self.isTerminalState(resultingState), None
        else:
            return self.agentPosition, reward, self.isTerminalState(self.agentPosition), None

    def reset(self):
        self.agentPosition = 0
        self.grid = np.zeros((self.m,self.n))
        return self.agentPosition, False


    def render(self):
        print('------------------------------------------')
        for row in self.grid:
            for col in row:
                if col == 0:
                    print('-', end='\t')
                elif col == 1:
                    print('X', end='\t')
            print('\n')
        print('------------------------------------------')


## First visit Monte Carlo Prediction

In [15]:
def MC_first_visit(loop=500, size=6, GAMMA=1.0, wind=[0, 0, 1, 2, 1, 0], part=0):


    grid = WindyGrid(size,size, wind)
    # GAMMA = 1.0

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    print(part)
    printV(V, grid)

## Del 1:

- Använd *first visit* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras värdefunktion om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med värdefunktion?
        - Prova med 𝛾=0,9, vad händer med värdefunktion?
    - 10x10
        - Ändra på vinden, vad händer med värdefunktion?
        - Prova med 𝛾=0,9, vad händer med värdefunktion?

In [16]:
MC_first_visit() # Default

# 1
# MC_first_visit(500, 6, 1.0, [1,1,2,3,2,1], 1) # Wind increase. It took forever to progress with wind changed, do not know why. It's the same for them all

# 2
MC_first_visit(500, 6, 0.5, [0, 0, 1, 2, 1, 0], 2) # Gamma 0.5
MC_first_visit(500, 6, 0.9, [0, 0, 1, 2, 1, 0], 3) # Gamma 0.9
MC_first_visit(500, 6, 0.95, [0, 0, 1, 2, 1, 0], 4) # Gamma 0.95

# 3 a
MC_first_visit(500, 8, 1.0, [0, 0, 1, 2, 1, 0], 5) # Default 8x8
# MC_first_visit(500, 8, 1.0, [1, 1, 2, 3, 2, 1], 6) # Wind increase 8x8
MC_first_visit(500, 8, 0.9, [0, 0, 1, 2, 1, 0], 7) # Gamma increase 8x8

# 3 b
MC_first_visit(500, 10, 1.0, [0, 0, 1, 2, 1, 0], 8) # Default 10x10
# MC_first_visit(500, 10, 1.0, [1, 1, 2, 3, 2, 1], 9) # Wind increase 10x10
# MC_first_visit(500, 10, 0.9, [1, 1, 2, 3, 2, 1], 10) # Wind increase 10x10



starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


0
-1523.21	-1522.51	-1509.65	-1496.10	-1481.04	-1460.69	

-1531.58	-1525.28	-1504.53	-1468.74	-1483.62	-1438.26	

-1526.62	-1528.63	-1494.21	-1451.67	-1469.20	-1362.64	

-1510.49	-1499.59	-1486.08	-1462.22	-1375.64	-1177.50	

-1478.52	-1464.60	-1451.62	-1474.60	0.00	-748.42	

-1463.66	-1457.46	-1433.27	0.00	-1218.66	-959.54	

--------------------
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


2
-2.00	-2.00	-2.00	-2.00	-2.00	-2.00	

-2.00	-2.00	-2.00	-2.00	-2.00	-2.00	

-2.00	-2.00	-2.00	-2.00	-2.00	-1.99	

-2.00	-2.00	-2.00	-2.00	-2.00	-1.95	

-2.00	-2.00	-2.00	-2.00	0.00	-1.68	

-2.00	-2.00	-2.00	0.00	-1.95	-1.93	

--------------------
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


3
-10.00	-10.00	-9.99	-9.98	-9.98	-9.94	

-10.00	-10.00	-9.99	-9.98	-9.96	-9.86	

-10.0

IndexError: ignored

## Exploring Start Monte Carlo

In [ ]:
def MC_exploring_starts(size=6, GAMMA=0.9, wind=[0, 0, 1, 2, 1, 0], part=0):

    grid = WindyGrid(size,size, wind)
    # GAMMA = 0.9

    # Initialize Q, returns, and pairs visited
    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]

    printQ(Q, grid)
    printPolicy(policy,grid)

## Del 2


- Använd  *exploring starts* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
        - Prova med 𝛾=0,9, vad händer med policyn?
    - 10x10
        - Ändra på vinden, vad händer med policyn?

In [ ]:
MC_exploring_starts(6, 0.9, [0, 0, 1, 2, 1, 0]) #  Default
MC_exploring_starts(6, 0.9, [0, 0, 1, 2, 1, 0], 1) #


## On-policy first visit Monte Carlo for $\varepsilon$-soft policies

In [ ]:
def MC_without_exploring_starts():
    grid = WindyGrid(6,6, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = 0.9
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()
        memory = []
        steps = 0
        while not done:
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

## Del 3
- Använd *without exploring starts* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
        - Prova med 𝛾=0,9, vad händer med policyn?
    - 10x10
        - Ändra på vinden, vad händer med policyn?

## Off-Policy Monte Carlo prediction

In [ ]:
def MC_off_policy_prediction():
    grid = WindyGrid(6,6, wind=[0,0,1,2,1,0])
    GAMMA = 0.9

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0

    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))

        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

## Del 4
- Använd *off-policy prediction* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
        - Prova med 𝛾=0,9, vad händer med policyn?
    - 10x10
        - Ändra på vinden, vad händer med policyn?